In [450]:
import tensorflow, torch, fitz
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, pipeline,DataCollatorWithPadding,Trainer
from datasets import Dataset, DatasetDict

In [452]:
path_name = r"C:\Users\STJ\Desktop\final_model_and_tokenizer"

tokenizer = GPT2Tokenizer.from_pretrained(path_name)
model = GPT2LMHeadModel.from_pretrained(path_name)

In [454]:
import pandas as pd
Data_Set = pd.read_csv("tdk_word_meaning_data.csv")
# Assuming Data_Set is your existing dataset with columns "madde" and "anlam"
Data_Set = Data_Set.rename(columns={"madde": "text", "anlam": "label"})


In [456]:
Data_Set

,text,label
0,acınılma,Acınılmak işi
1,acube,Tuhaf kimse
2,açık fikirlilik,Açık fikirli olma durumu
3,adaklanma,Nişanlanma
4,adaletlilik,Adaletli olma durumu
...,...,...
92404,aile mahkemesi,Her ilde ve merkez nüfusu yüz binin üzerindeki...
92405,alicengiz oyunu,"Kurnazca ve haince hazırlanmış hile, düzen, oyun"
92406,özçekim,Kamerası olan cep telefonu veya tablet kullana...
92407,selfie,bk. özçekim


In [458]:
dataset_dict = Data_Set.to_dict('records')

In [460]:
dataset_dict

[{'text': 'acınılma', 'label': 'Acınılmak işi'},
 {'text': 'acube', 'label': 'Tuhaf kimse'},
 {'text': 'açık fikirlilik', 'label': 'Açık fikirli olma durumu'},
 {'text': 'adaklanma', 'label': 'Nişanlanma'},
 {'text': 'adaletlilik', 'label': 'Adaletli olma durumu'},
 {'text': 'addetme', 'label': 'Sayma'},
 {'text': 'addolunma', 'label': 'Sayılma'},
 {'text': 'adli polis',
  'label': 'Adliye içerisinde güvenliği sağlayıp cumhuriyet savcısına bağlı olarak çalışan kolluk gücü'},
 {'text': 'aforozlama', 'label': 'Aforozlamak işi'},
 {'text': 'afsunlama', 'label': 'Büyüleme'},
 {'text': 'afsunlanma', 'label': 'Büyülenme'},
 {'text': 'afyonlama', 'label': 'Afyonlamak işi'},
 {'text': 'afyonlanma', 'label': 'Afyonlanmak işi'},
 {'text': 'ağaçlama', 'label': 'Ağaçlamak işi'},
 {'text': 'ağaçlandırılma', 'label': 'Ağaçlandırılmak işi'},
 {'text': 'ağaçlanma', 'label': 'Ağaçlanmak işi'},
 {'text': 'ağalanma', 'label': 'Ağalanmak durumu'},
 {'text': 'ağdalanma', 'label': 'Ağdalanmak işi'},
 {'text

In [462]:
data_set

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [464]:
Train_DataSet = Dataset.from_dict(Data_Set[:200]);
Validation_DataSet = Dataset.from_dict(Data_Set[200:1000]);

In [466]:
Train_DataSet

Dataset({
    features: ['text', 'label'],
    num_rows: 200
})

In [468]:
Validation_DataSet

Dataset({
    features: ['text', 'label'],
    num_rows: 800
})

In [470]:
dataset_dict = DatasetDict({
    'train': Train_DataSet,
    'validation': Validation_DataSet,
})

In [472]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 800
    })
})

In [474]:
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel, GPT2Tokenizer
from datasets import Dataset, DatasetDict

# Load pretrained model and tokenizer
path_name = r"C:\Users\STJ\Desktop\final_model_and_tokenizer"
tokenizer = GPT2Tokenizer.from_pretrained(path_name, add_prefix_space=True)

def tokenize_function(examples):
    text = examples["text"]
    label = examples["label"]  # Örnek olarak "label" sütununun varlığından ve içeriğinden bahsetmişsiniz.

    tokenizer.truncation_side = "left"
    
    # Tokenize text
    tokenized_inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Tokenize label
    tokenized_label = tokenizer(label, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Combine inputs and label
    tokenized_inputs["labels"] = tokenized_label["input_ids"].squeeze()  # Modelin öğrenmesi için "labels" olarak belirtilmiş bir girdi

    return tokenized_inputs

# Eğer tokenizer'ınızda padding token yoksa ekleyin
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenized_dataset = dataset_dict.map(tokenize_function,batched=True)
tokenized_dataset

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
})

In [476]:
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    
    # Generate text
    output = model.generate(input_ids, max_length=max_length, attention_mask=attention_mask, 
                            pad_token_id=tokenizer.pad_token_id, num_return_sequences=1)
    
    # Decode and return generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text


In [478]:
prompt = "nefret ?"
generated_text = generate_text(prompt, max_length=50)
print("Generated Text:", generated_text)


Generated Text:  nefret?
Bu kadar basit.
Bu kadar basit.
Bu kadar basit.
Bu kadar basit.
Bu kadar basit.
Bu kadar basit.
Bu kadar basit.
Bu kadar basit.
Bu kadar basit.
Bu kadar


In [479]:
from transformers import GPT2Config

config = GPT2Config(
    vocab_size=tokenizer.vocab_size,  # Replace with your tokenizer's vocab size
    n_positions=512,  # Maximum sequence length
    task_specific_params={"text-generation": {"do_sample": True, "max_length": 100}},
)

# Load pretrained model
model = GPT2LMHeadModel(config=config)

In [481]:
from transformers import TrainingArguments, IntervalStrategy

lr = 1e-3  # Öğrenme oranı (0.001)
batch_size = 4
num_epochs = 10

training_args = TrainingArguments(
    output_dir="text_gen",  # Modelin kaydedileceği dizin
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",  # Değerlendirme stratejisi
    logging_strategy="epoch",  # Günlükleme stratejisi
    save_strategy="epoch",  # Kaydetme stratejisi
    load_best_model_at_end=True,  # Eğitim sonunda en iyi modeli yükle
)

C:\Users\STJ\anaconda3\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [484]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer = tokenizer,
    data_collator=data_collator,
)
trainer.train()

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [43]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import pandas as pd

# Load pretrained model and tokenizer
path_name = r"C:\Users\STJ\Desktop\final_model_and_tokenizer"
tokenizer = GPT2Tokenizer.from_pretrained(path_name, add_prefix_space=True)

# Check if tokenizer has a pad_token and add it if not
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2LMHeadModel.from_pretrained(path_name)

# Load your dataset
Data_Set = pd.read_csv("tdk_word_meaning_data.csv")
Data_Set = Data_Set.rename(columns={"madde": "text", "anlam": "label"})
Data_Set = Data_Set[0:10]
Train_DataSet = Dataset.from_dict(Data_Set)
Validation_DataSet = Dataset.from_dict(Data_Set)
dataset_dict = DatasetDict({
    'train': Train_DataSet,
    'validation': Validation_DataSet,
})

def tokenize_function(examples):
    text = examples["text"]
    label = examples["label"]

    # Tokenize text
    tokenized_inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Tokenize label
    tokenized_label = tokenizer(label, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Ensure labels are correctly formatted for model training
    tokenized_inputs["label"] = tokenized_label["input_ids"].squeeze()
    
    return tokenized_inputs

# Map tokenize_function to tokenized datasets
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

# Print tokenized datasets to inspect keys
print(tokenized_datasets)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_steps=100,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10
    })
})


C:\Users\STJ\anaconda3\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


IndexError: index out of range in self

In [515]:
example = tokenized_datasets['train'][0]
print("Input IDs:", example["input_ids"])
print("Attention Mask:", example["attention_mask"])
print("Label (Next Token):", example["labels"])
print("Label (Next Token):", example["label"])

Input IDs: [1873, 266, 7267, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257]
Attention Mask: [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Label (Next Token): [5408, 266, 8223, 3656, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257]
Label (Next Token): [5408, 266, 8223, 3656, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257]


In [517]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
})

In [231]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import pandas as pd

# Load pretrained model and tokenizer
path_name = r"C:\Users\STJ\Desktop\final_model_and_tokenizer"
tokenizer = GPT2Tokenizer.from_pretrained(path_name, add_prefix_space=True)

# Check if tokenizer has a pad_token and add it if not
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))


model = GPT2LMHeadModel.from_pretrained(path_name)

# Load your dataset as DataFrame
Data_Set = pd.read_csv("tdk_word_meaning_data.csv")
Data_Set = Data_Set.rename(columns={"madde": "text", "anlam": "label"})

# Concatenate 'text' and 'label' columns into 'text' column
Data_Set['text'] = Data_Set['text'].astype(str) + " " + Data_Set['label'].astype(str)

# Create tokenized_datasets dictionary
non_tokenized_datasets = {
    'train': Data_Set[:1000],
    'validation': Data_Set[1000:2000]
}

# Create Dataset object from DataFrame
dataset = non_tokenized_datasets.from_pandas(Data_Set)

AttributeError: 'dict' object has no attribute 'from_pandas'

In [364]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [378]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import pandas as pd

# Load pretrained model and Tokenizer
path_name = r"C:\Users\STJ\Desktop\final_model_and_tokenizer"
tokenizer = GPT2Tokenizer.from_pretrained(path_name, add_prefix_space=True)

# Check if tokenizer has a pad_token and add it if not
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

model = GPT2LMHeadModel.from_pretrained(path_name)

# Load your dataset as DataFrame
Data_Set = pd.read_csv("tdk_word_meaning_data.csv")
Data_Set = Data_Set.rename(columns={"madde": "text", "anlam": "label"})

# Concatenate 'text' and 'label' columns into 'text' column
Data_Set['text'] = Data_Set['text'].astype(str) + " " + Data_Set['label'].astype(str)

# Convert pandas DataFrames to datasets
train_dataset = Dataset.from_pandas(Data_Set[:300])
validation_dataset = Dataset.from_pandas(Data_Set[300:600])

# Create DatasetDict object
non_tokenized_datasets = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset
})

# Tokenization function
def tokenize_function(examples):
    # Correct truncation_side attribute name
    tokenizer.truncation_side = "left"
    # Tokenize text with padding
    tokenized_inputs = tokenizer(examples["text"], return_tensors="pt", truncation=True, padding=True, max_length=128)
    return tokenized_inputs

# Map tokenize_function to tokenized datasets
tokenized_datasets = non_tokenized_datasets.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

peft_config = LoraConfig(
    task_type="text-generation",
    r=4,  # Örnek bir başka parametre
    lora_alpha=32,
    lora_dropout=0.01,
    target_modules=['q_lin']
)

def compute_acc(p):
    predictions, labels = p
    predicitons = np.argmax(predicitons,axis=1)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    save_steps=100,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

# Train the model
trainer.train()


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

ValueError: Target modules {'q_lin'} not found in the base model. Please check the target modules and try again.

In [314]:
for key, value in tokenized_datasets.items():
    if isinstance(value, (dict, list)):
        print(f"Key '{key}' has a nested structure.")
    else:
        print(f"Key '{key}' is one-dimensional.")

# Or, simply check the type of each value
for key, value in tokenized_datasets.items():
    print(f"Key '{key}' has type '{type(value).__name__}'.")

Key 'train' is one-dimensional.
Key 'validation' is one-dimensional.
Key 'train' has type 'Dataset'.
Key 'validation' has type 'Dataset'.


In [382]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import pandas as pd
import random
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

# Load pretrained model and tokenizer
path_name = r"C:\Users\STJ\Desktop\final_model_and_tokenizer"
tokenizer = GPT2Tokenizer.from_pretrained(path_name, add_prefix_space=True)

# Check if tokenizer has a pad_token and add it if not
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

model = GPT2LMHeadModel.from_pretrained(path_name)

# Load your dataset as DataFrame
Data_Set = pd.read_csv("tdk_word_meaning_data.csv")
Data_Set = Data_Set.rename(columns={"madde": "text", "anlam": "label"})

# Concatenate 'text' and 'label' columns into 'text' column
Data_Set['text'] = Data_Set['text'].astype(str) + " " + Data_Set['label'].astype(str)

# Create Dataset object from DataFrame
dataset = Dataset.from_pandas(Data_Set)

def tokenize_function(examples):
    # Tokenize text
    tokenized_inputs = tokenizer(examples["text"], return_tensors="pt", truncation=True, padding='max_length', max_length=128)
    
    # Filter out padding tokens and adjust attention mask
    for i in range(len(tokenized_inputs["input_ids"])):
        input_ids = tokenized_inputs["input_ids"][i]
        attention_mask = tokenized_inputs["attention_mask"][i]

        # Find the position of the first padding token
        first_pad_token_idx = (input_ids == tokenizer.pad_token_id).nonzero(as_tuple=True)[0]

        # If padding token is found, adjust input_ids and attention_mask
        if first_pad_token_idx.numel() > 0:
            first_pad_token_idx = first_pad_token_idx.item()
            tokenized_inputs["input_ids"][i] = input_ids[:first_pad_token_idx]
            tokenized_inputs["attention_mask"][i] = attention_mask[:first_pad_token_idx]
    
    return tokenized_inputs



# Map tokenize_function to tokenized datasets

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Specify the subset size
DataSet_SubSize = 1000

# Shuffle the dataset and select the first DataSet_SubSize examples
tokenized_dataset = tokenized_dataset.shuffle(seed=42)
tokenized_dataset = tokenized_dataset.select(range(DataSet_SubSize))

# Debug: Print total number of examples in tokenized_dataset
print("Total number of examples in tokenized_dataset:", len(tokenized_dataset))

# Debug: Print some examples from tokenized_dataset
print("First example in tokenized_dataset:", tokenized_dataset[0])

# Split the tokenized dataset into train and validation sets
train_size = int(0.7 * len(tokenized_dataset))
validation_size = len(tokenized_dataset) - train_size

tokenized_datasets = {
    'train': tokenized_dataset[:train_size],
    'validation': tokenized_dataset[train_size:]
}

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    save_steps=100,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

# Train the model
trainer.train()


Map:   0%|          | 0/92409 [00:00<?, ? examples/s]

RuntimeError: a Tensor with 121 elements cannot be converted to Scalar

In [308]:
for key, value in tokenized_datasets.items():
    if isinstance(value, (dict, list)):
        print(f"Key '{key}' has a nested structure.")
    else:
        print(f"Key '{key}' is one-dimensional.")

# Or, simply check the type of each value
for key, value in tokenized_datasets.items():
    print(f"Key '{key}' has type '{type(value).__name__}'.")

Key 'train' has a nested structure.
Key 'validation' has a nested structure.
Key 'train' has type 'dict'.
Key 'validation' has type 'dict'.


In [380]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
